In [1]:
import json
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import KFold

In [2]:
training= pd.read_csv("ABIDE_train_images_all.csv")
training

,SUB_ID,DX_GROUP,AGE_AT_SCAN,SEX,FIQ,VIQ,PIQ,FIQ_TEST_TYPE,VIQ_TEST_TYPE,PIQ_TEST_TYPE,PATH
0,51456,1,55.40,M,126.0,118.0,128.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,1,22.90,M,107.0,119.0,93.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,1,39.20,M,93.0,80.0,108.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,1,22.80,M,106.0,94.0,118.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,1,34.60,F,133.0,135.0,122.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...,...,...,...,...,...,...,...
1080,50624,1,11.08,F,90.0,91.0,89.0,DAS_II_SA,DAS_II_SA,DAS_II_SA,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1081,50625,1,7.00,M,99.0,90.0,98.0,DAS_II_SA,DAS_II_SA,DAS_II_SA,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1082,50626,1,11.08,M,61.0,66.0,60.0,DAS_II_SA,DAS_II_SA,DAS_II_SA,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1083,50627,1,9.50,F,88.0,103.0,84.0,DAS_II_SA,DAS_II_SA,DAS_II_SA,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [3]:
training=training.drop(["DX_GROUP","AGE_AT_SCAN","FIQ_TEST_TYPE","VIQ_TEST_TYPE","PIQ_TEST_TYPE"],axis=1)

**FIQ**

In [5]:
df=training[training["FIQ"].notna()].reset_index(drop=True)
df

,SUB_ID,SEX,FIQ,VIQ,PIQ,PATH
0,51456,M,126.0,118.0,128.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,M,107.0,119.0,93.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,M,93.0,80.0,108.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,M,106.0,94.0,118.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,F,133.0,135.0,122.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...,...,...
1045,50624,F,90.0,91.0,89.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1046,50625,M,99.0,90.0,98.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1047,50626,M,61.0,66.0,60.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1048,50627,F,88.0,103.0,84.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [6]:
df=df[df["FIQ"]>=0].reset_index(drop=True)
df=df[["SUB_ID","FIQ","SEX","PATH"]]
df['FIQ']=df["FIQ"].astype('int64')
df

,SUB_ID,FIQ,SEX,PATH
0,51456,126,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,107,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,93,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,106,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,133,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...
1009,50624,90,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1010,50625,99,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1011,50626,61,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1012,50627,88,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [8]:
df.to_csv(r'folderlist2/all_fiq_data_new.list',sep=' ',index=False,header=None)

**PIQ**

In [9]:
df=training[training["PIQ"].notna()].reset_index(drop=True)
df

,SUB_ID,SEX,FIQ,VIQ,PIQ,PATH
0,51456,M,126.0,118.0,128.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,M,107.0,119.0,93.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,M,93.0,80.0,108.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,M,106.0,94.0,118.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,F,133.0,135.0,122.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...,...,...
929,50624,F,90.0,91.0,89.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
930,50625,M,99.0,90.0,98.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
931,50626,M,61.0,66.0,60.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
932,50627,F,88.0,103.0,84.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [10]:
df=df[df["PIQ"]>=0].reset_index(drop=True)
df=df[["SUB_ID","PIQ","SEX","PATH"]]
df['PIQ']=df["PIQ"].astype('int64')
df

,SUB_ID,PIQ,SEX,PATH
0,51456,128,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,93,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,108,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,118,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,122,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...
907,50624,89,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
908,50625,98,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
909,50626,60,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
910,50627,84,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [11]:
df.to_csv(r'folderlist2/all_piq_data_new.list',sep=' ',index=False,header=None)

**VIQ**

In [12]:
df=training[training["VIQ"].notna()].reset_index(drop=True)
df

,SUB_ID,SEX,FIQ,VIQ,PIQ,PATH
0,51456,M,126.0,118.0,128.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,M,107.0,119.0,93.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,M,93.0,80.0,108.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,M,106.0,94.0,118.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,F,133.0,135.0,122.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...,...,...
911,50624,F,90.0,91.0,89.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
912,50625,M,99.0,90.0,98.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
913,50626,M,61.0,66.0,60.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
914,50627,F,88.0,103.0,84.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [13]:
df=df[df["VIQ"]>=0].reset_index(drop=True)
df=df[["SUB_ID","VIQ","SEX","PATH"]]
df['VIQ']=df["VIQ"].astype('int64')
df

,SUB_ID,VIQ,SEX,PATH
0,51456,118,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,119,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,80,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,94,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,135,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...
893,50624,91,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
894,50625,90,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
895,50626,66,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
896,50627,103,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [14]:
df.to_csv(r'folderlist2/all_viq_data_new.list',sep=' ',index=False,header=None)